# Part 0:
## import everything
Run the cell below

In [ ]:
import os
import glob
import numpy as np
from platform import system as OS
import pandas as pd
import scipy.stats
import math
import datetime
from copy import deepcopy
import matplotlib.cm as cm
import warnings
warnings.filterwarnings("ignore")
import sys
import pickle
import string
import matplotlib as mpl
import matplotlib.pyplot as plt
import PIL
import PyPDF2 as ppdf
from scipy import stats
from scipy.ndimage.filters import gaussian_filter as smooth
import matplotlib.animation as animation
import matplotlib.backends.backend_pdf
import mpl_toolkits.axes_grid1.inset_locator as inset
from matplotlib.ticker import FormatStrFormatter, MaxNLocator, ScalarFormatter, FuncFormatter, PercentFormatter
from matplotlib.patches import ConnectionPatch, FancyArrowPatch
from set_rc_params import set_rc_params
import ROOT


if "__file__" not in dir():
    %matplotlib inline
    %config InlineBackend.close_figures = False

    root=ROOT.root
    
    ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
    CommonNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
    CWD=os.getcwd()
    os.chdir(CommonNoteBookesPath)
    %run UtilityTools.ipynb
    %run Animal_Tags.ipynb
    %run loadRat_documentation.ipynb
    %run Lesion_Size.ipynb
    %run plotRat_documentation_1_GeneralBehavior.ipynb
    %run plotRat_documentation_3_KinematicsInvestigation.ipynb
    %run RunBatchRat_3_CompareGroups.ipynb
    %run BatchRatBehavior.ipynb
    currentNbPath=os.path.join(os.path.split(ThisNoteBookPath)[0],'LesionPaper','CorrectTrialCurve.ipynb')
    %run $currentNbPath

    os.chdir(CWD)

    logging.getLogger().setLevel(logging.ERROR)
    
    param={
        "goalTime":7,#needed for pavel data only
        "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
        "maxTrialDuration":15,
        "interTrialDuration":10,#None pavel
        "endTrial_frontPos":30,
        "endTrial_backPos":55, 
        "endTrial_minTimeSec":4,
        "cameraSamplingRate":25, #needed for new setup    

        "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
        "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
        "nbJumpMax":100,#200 pavel
        "binSize":0.25,
        #parameters used to preprocess (will override the default parameters)
    }
    Y1,Y2=param['treadmillRange']

    print('os:',OS(),'\nroot:',root,'\nImport successful!')

---
---


# part 1:

# DEFINITIONS

### If you don't know what to do, move to part 2

In [ ]:
def add_panel_caption(axes: tuple, offsetX: tuple, offsetY: tuple, **kwargs):
    """
    This function adds letter captions (a,b,c,d) to Axes in axes
    at top left, with the specified offset, in RELATIVE figure coordinates
    """
    assert len(axes)==len(offsetX)==len(offsetY), 'Bad input!'
    
    fig=axes[0].get_figure()
    fbox=fig.bbox
    for ax,dx,dy,s in zip(axes,offsetX,offsetY,string.ascii_uppercase):
        axbox=ax.get_window_extent()
    
        ax.text(x=(axbox.x0/fbox.xmax)-abs(dx), y=(axbox.y1/fbox.ymax)+abs(dy),
                s=s,fontweight='extra bold', fontsize=10, ha='left', va='center',
               transform=fig.transFigure,**kwargs)

---

plot learninig curve

In [ ]:
def _percent_trial_over_six(data):
    N=len(data.entranceTime)
    
#     n=np.sum(data.entranceTime >= 7) - np.sum(data.entranceTime==data.maxTrialDuration[1])
    n=np.sum(np.logical_and(data.entranceTime > 6,data.entranceTime < 10))

    return (n/N)*100

def plot_panel_A(ax1,ax2):
    #learining curve
    PerfParam=_percent_trial_over_six.__name__
    
    profile=   {'Type':'Good',
                'rewardType':'Progressive',
                'initialSpeed':['10'],
                'Speed':'10',
                'Tag':['Control', 'Control-Sharp']}
    animalList= batch_get_animal_list(root,profile)
    NSession=30
    data=np.ones((NSession,len(animalList)))*np.nan
    for i,animal in enumerate(animalList):
        out=animal_learning_stats(root, animal, PerfParam=PerfParam,profile=profile,redo=False)
        if len(out) < NSession:continue
        data[:,i]=out[:NSession]
    
    x=np.arange(1,NSession+1)
    y=np.nanmedian(data,axis=1)
    Gline=ax1.plot(x, y,color='xkcd:black')
    ax1.fill_between(x,y1=np.nanpercentile(data,25,axis=1),y2=np.nanpercentile(data,75,axis=1),
                    color='xkcd:gray')
    out=animal_learning_stats(root, 'Rat304', PerfParam=PerfParam,profile=profile)[:NSession]
    Eline=ax1.plot(x, out,'v',lw=1,ms=2,mfc='white',mec='k',markeredgewidth=.5)

    ax1.legend([*Gline,*Eline],[f'Group\n($n={np.sum(~np.isnan(data[1,:]))}$)','Example'],
               loc='lower right', fontsize=5,frameon=False,markerscale=2)
    ax1.set_xlim([0,NSession+1])
    ax1.spines['bottom'].set_bounds(0,NSession)
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.set_ylim([0,85])
    ax1.set_yticks(np.arange(0,81,10))
    ax1.spines['left'].set_bounds(0,80)
    ax1.yaxis.set_major_formatter(PercentFormatter())
    ax1.tick_params('both', labelsize='xx-small',)
    ax1.set_xlabel('Learning session #',labelpad=0,fontsize='xx-small')
    ax1.set_ylabel('Trials with $6<ET<10$' ,labelpad=0,fontsize='xx-small')
    
    #second panel
    #learining curve
    PerfParam='standard deviation of entrance time'
    
    profile=   {'Type':'Good',
                'rewardType':'Progressive',
                'initialSpeed':['10'],
                'Speed':'10',
                'Tag':['Control', 'Control-Sharp']}
    animalList= batch_get_animal_list(root,profile)
    NSession=30
    data=np.ones((NSession,len(animalList)))*np.nan
    for i,animal in enumerate(animalList):
        out=animal_learning_stats(root, animal, PerfParam=PerfParam,profile=profile,redo=False)
        if len(out) < NSession:continue
        data[:,i]=out[:NSession]
    
    x=np.arange(1,NSession+1)
    y=np.nanmedian(data,axis=1)
    Gline=ax2.plot(x, y,color='xkcd:black')
    ax2.fill_between(x,y1=np.nanpercentile(data,25,axis=1),y2=np.nanpercentile(data,75,axis=1),
                    color='xkcd:gray')
    out=animal_learning_stats(root, 'Rat304', PerfParam=PerfParam,profile=profile)[:NSession]
    Eline=ax2.plot(x, out,'v',lw=1,ms=2,mfc='white',mec='k',markeredgewidth=.5)

    ax2.set_xlim([0,NSession+1])
    ax2.spines['bottom'].set_bounds(0,NSession)
    ax2.spines['top'].set_visible(False)
    ax2.spines['right'].set_visible(False)
    ax2.set_ylim([1,5])
    ax2.set_yticks(np.arange(1,5.1,1))
    ax2.spines['left'].set_bounds(1,5)
    ax2.tick_params('both', labelsize='xx-small',)
    ax2.set_xlabel('Learning session #',labelpad=0,fontsize='xx-small')
    ax2.set_ylabel('$SD_{ET}$ (s)' ,labelpad=0,fontsize='xx-small')

    
    
    return data, animalList

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    fig=plt.figure(figsize=(10,5))
    ax1=fig.add_subplot(111)
    
    a=plot_panel_A(ax1)

------



------

# part 2:

# GENERATING THE FIGURE

Plotting the figure

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc_params()
    figsize=(3,1)
    fig=plt.figure(figsize=figsize,dpi=600)
    
   

    ##########################################
    # 1: task rules
    gs1   =fig.add_gridspec(nrows=1, ncols=2, left=0, bottom=0, right=1, top=1)
    ax1   =fig.add_subplot(gs1[0])
    ax2   =fig.add_subplot(gs1[1])
    
    plot_panel_A(ax1,ax2)

    
    
#     fig.align_ylabels([ax2,ax4,ax6])
    #############################################
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    AXES=(ax1,ax2)
    OFFX=np.array([.07]*len(AXES))
    OFFY=np.array([.05]*len(AXES))
    
    add_panel_caption(axes=AXES, offsetX=OFFX, offsetY=OFFY)
    
    fig.savefig(os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','CorrectTrialCurve.pdf'),
                format='pdf', bbox_inches='tight')
        
    plt.show()
    plt.close('all')
    matplotlib.rcdefaults()